In [1]:
import sys

import rics

# Print relevant versions
print(f"{rics.__version__=}")
print(f"{sys.version=}")
!git log --pretty=oneline --abbrev-commit -1

rics.__version__='0.14.0'
sys.version='3.9.12 (main, Jun  1 2022, 11:38:51) \n[GCC 7.5.0]'
e55e660 (HEAD -> master, origin/master, origin/HEAD) Relax perftest condition


# Select using `or` vs `is None` vs `if [bool]`
Null dereferencing.

## Load data
Load some representative data we need for testing (that doesn't change between test cases or candidates).

In [2]:
import random

test_data = random.choices(
    [
        [1, 2, 3],
        None,
        # [],
    ],
    k=100_000,
)

## Define the test procedure
Define the test case we want to evaluate.

In [3]:
def run(func):
    for arg in test_data:
        func(arg)

## Define candidates
Define the functions or classes we're testing. Add to `candidates` list.

In [4]:
fallback_value = ["foo", "bar"]


def select_or(arg):
    return arg or fallback_value


def select_is_none(arg):
    return fallback_value if arg is None else arg


def select_bool(arg):
    return arg if arg else fallback_value


candidates = [select_or, select_is_none, select_bool]

## Sample output and verification
Output sample result, make sure candidates outpat equivalent values.

In [5]:
reference_output = candidates[0](test_data[0])
assert reference_output is not None
print("Reference output:", reference_output)

for cand in candidates[1:]:
    cand_output = cand(test_data[0])
    assert (
        reference_output == cand_output
    ), f"Bad candidate: {cand}. {cand_output=} != {reference_output=}"

Reference output: ['foo', 'bar']


## Run performance comparison

In [6]:
from rics.utility.misc import tname

In [7]:
for cand in candidates:
    print(f"{tname(cand)}:")
    %timeit -r 100 -n 100 run(cand)
    print("=" * 80)

select_or:
6.44 ms ± 523 µs per loop (mean ± std. dev. of 100 runs, 100 loops each)
select_is_none:
7.25 ms ± 1.61 ms per loop (mean ± std. dev. of 100 runs, 100 loops each)
select_bool:
7.39 ms ± 1.56 ms per loop (mean ± std. dev. of 100 runs, 100 loops each)


# Conclusion
Use whatever is more readable.